<a id=contents></a>

# Model building
## What are we trying to predict?


[1. Target variable and choice of evaluation metric](#target)

[2. Class imbalance / sampling solution / distribution](#samp)

[3. Train-test split and model transformation](#ttsplit)

[4. Choice of model architectures](#selection)

[4.1 Model 1](#one)

[4.2 Model 2](#two)

[4.2 Model 3](#three)

[4.2 Model 4](#four)

[4.2 Model 5](#five)

[7. Conclusions and model comparison table](#conc)

In [9]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("darkgrid")

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import f1_score

from sklearn_crfsuite import CRF, scorers, metrics
from sklearn
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id=target ><a/> 

## 1. ETL of data and Train-Test Split
    
[LINK to table of contents](#contents)

In [3]:
feature_df = pd.read_pickle('feature_based_data/final_feature_data.pkl')
feature_y = feature_df.Tag
feature_X = feature_df.drop(columns=['Tag'])

x_train, x_test, y_train, y_test = train_test_split(feature_X, feature_y, test_size=.2, random_state=12345)

In [8]:
x_train.shape

(52928, 13)

In [11]:
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [ ]:
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)


<a id=samp ><a/> 

## 2.  Class imbalance / sampling solution / distribution
    
[LINK to table of contents](#contents)

<a id=ttsplit ><a/> 

## 3. Train-test split and model transformation
   
[LINK to table of contents](#contents)

<a id=selection></a>

## 4. Choice of model architectures

[LINK to table of contents](#contents)

<a id=one ><a/> 

## 4.1 Model 1
    
[LINK to table of contents](#contents)

<a id=two ><a/> 

## 4.2 Model 2
    
[LINK to table of contents](#contents)

<a id=three ><a/> 

## 4.3 Model 3
    
[LINK to table of contents](#contents)

<a id=four ><a/> 

## 4.4 Model 4
    
[LINK to table of contents](#contents)

<a id=five ><a/> 

## 4.5 Model 5
    
[LINK to table of contents](#contents)

<a id=conc ><a/> 

## 7. Conclusions and model comparison table
    
[LINK to table of contents](#contents)